In [1]:
!./my_repocontroll.sh

remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/class-snct-rikitakelab/ParticleFluidPy
 * branch            master     -> FETCH_HEAD
Updating 48aa505..88e4aeb
Fast-forward
 algorithm.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)
[master 157e664] update submodule
 1 file changed, 1 insertion(+), 1 deletion(-)


In [2]:
from __future__ import print_function

from numpy import ones_like, mgrid, sqrt

from pysph.base.utils import get_particle_array
from pysph.solver.application import Application
from pysph.sph.scheme import WCSPHScheme


class EllipticalDrop(Application):
    def initialize(self):
        self.co = 1400.0
        self.ro = 1.0
        self.hdx = 1.3
        self.dx = 0.025
        self.alpha = 0.1

    def create_scheme(self):
        s = WCSPHScheme(
            ['fluid'], [], dim=2, rho0=self.ro, c0=self.co,
            h0=self.dx*self.hdx, hdx=self.hdx, gamma=7.0, alpha=0.1, beta=0.0
        )
        dt = 5e-6
        tf = 0.0076
        s.configure_solver(dt=dt, tf=tf)
        with open('scheme.pickle', mode='wb') as f:
            pickle.dump(s, f)      
        return s

    def create_particles(self):
        """Create the circular patch of fluid."""
        dx = self.dx
        hdx = self.hdx
        ro = self.ro
        name = 'fluid'
        x, y = mgrid[-1.05:1.05+1e-4:dx, -1.05:1.05+1e-4:dx]
        x = x.ravel()
        y = y.ravel()

        m = ones_like(x)*dx*dx*ro
        h = ones_like(x)*hdx*dx
        rho = ones_like(x) * ro
        u = -100*x
        v = 100*y

        # remove particles outside the circle
        indices = []
        for i in range(len(x)):
            if sqrt(x[i]*x[i] + y[i]*y[i]) - 1 > 1e-10:
                indices.append(i)

        pa = get_particle_array(x=x, y=y, m=m, rho=rho, h=h, u=u, v=v,
                                name=name)
        pa.remove_particles(indices)

        print("Elliptical drop :: %d particles"
              % (pa.get_number_of_particles()))

        self.scheme.setup_properties([pa])
        with open('particle.pickle', mode='wb') as f:
            pickle.dump(pa, f)        
        return [pa]


if __name__ == '__main__':
    import pickle
    
    app = EllipticalDrop()
    app.run()

Elliptical drop :: 5025 particles
Generating output in /home/ec2-user/SageMaker/PySPH/__main___output
Compiling code at: /home/ec2-user/.pysph/source/py3.6-linux-x86_64/m_8d82621ec30b3bfde1aaaa37c8a65f4f.pyx
running build_ext
cythoning /home/ec2-user/.pysph/source/py3.6-linux-x86_64/m_8d82621ec30b3bfde1aaaa37c8a65f4f.pyx to /home/ec2-user/.pysph/source/py3.6-linux-x86_64/m_8d82621ec30b3bfde1aaaa37c8a65f4f.cpp
building 'm_8d82621ec30b3bfde1aaaa37c8a65f4f' extension
creating /home/ec2-user/.pysph/source/py3.6-linux-x86_64/build/temp.linux-x86_64-3.6
creating /home/ec2-user/.pysph/source/py3.6-linux-x86_64/build/temp.linux-x86_64-3.6/home
creating /home/ec2-user/.pysph/source/py3.6-linux-x86_64/build/temp.linux-x86_64-3.6/home/ec2-user
creating /home/ec2-user/.pysph/source/py3.6-linux-x86_64/build/temp.linux-x86_64-3.6/home/ec2-user/.pysph
creating /home/ec2-user/.pysph/source/py3.6-linux-x86_64/build/temp.linux-x86_64-3.6/home/ec2-user/.pysph/source
creating /home/ec2-user/.pysph/source/

In [35]:
from __future__ import print_function
from pysph.solver.application import Application

import pickle
    
class EllipticalDrop(Application):
    def create_scheme(self):
        with open('scheme.pickle', mode='rb') as f:
            s=pickle.load(f)      
        return s

    def create_particles(self):
        with open('particle.pickle', mode='rb') as f:
            pa=pickle.load(f)        
        return [pa]


if __name__ == '__main__':
    app = EllipticalDrop()
    app.run()

Generating output in /home/ec2-user/SageMaker/PySPH/__main___output
Precompiled code from: /home/ec2-user/.pysph/source/py2.7-linux-x86_64/m_30bb3da0068142637bd63ff68cf26310.pyx
No of particles:
----------------------------------------------------------------------
  fluid: 5025
----------------------------------------------------------------------
Setup took: 0.10446 secs
100%
Run took: 16.25209 secs


In [36]:
from numpy import ones_like, mgrid, sqrt

from pysph.base.utils import get_particle_array
from pysph.sph.scheme import WCSPHScheme

co = 1400.0
ro = 1.0
hdx = 1.3
dx = 0.025
alpha = 0.1
s=None

def create_scheme():
    global s
    s = WCSPHScheme(
        ['fluid'], [], dim=2, rho0=ro, c0=co,
        h0=dx*hdx, hdx=hdx, gamma=7.0, alpha=0.1, beta=0.0
    )
    dt = 5e-6
    tf = 0.0076
    s.configure_solver(dt=dt, tf=tf)
    with open('scheme.pickle', mode='wb') as f:
        pickle.dump(s, f)   
            
def create_particles():
    """Create the circular patch of fluid."""
    name = 'fluid'
    x, y = mgrid[-1.05:1.05+1e-4:dx, -1.05:1.05+1e-4:dx]
    x = x.ravel()
    y = y.ravel()

    m = ones_like(x)*dx*dx*ro
    h = ones_like(x)*hdx*dx
    rho = ones_like(x) * ro
    u = -100*x
    v = 100*y

    # remove particles outside the circle
    indices = []
    for i in range(len(x)):
        if sqrt(x[i]*x[i] + y[i]*y[i]) - 1 > 1e-10:
            indices.append(i)

    pa = get_particle_array(x=x, y=y, m=m, rho=rho, h=h, u=u, v=v,
                            name=name)
    pa.remove_particles(indices)

    s.setup_properties([pa])
    print("Elliptical drop :: %d particles"
          % (pa.get_number_of_particles()))

    with open('particle.pickle', mode='wb') as f:
        pickle.dump(pa, f)     
        
create_scheme()
create_particles()

Elliptical drop :: 5025 particles


In [3]:
paramet={
'co' : 1400.0,
'ro' : 1.0,
'hdx' : 1.3,
'dx' : 0.025,
'alpha' : 0.1,
'dt' : 5e-6,
'tf' : 0.0076,
}
    
#Test用のパラメータを作成
model_path='model/'
param_path = 'par.json'
import json

with open(param_path, mode='w') as f:
    json.dump(paramet,f)

In [4]:
from __future__ import print_function
from pysph.solver.application import Application

from numpy import ones_like, mgrid, sqrt

from pysph.base.utils import get_particle_array
from pysph.base.utils import get_particle_array_wcsph
from pysph.base.kernels import CubicSpline

from pysph.solver.solver import Solver
from pysph.sph.integrator import EPECIntegrator,EulerIntegrator
from pysph.sph.integrator_step import WCSPHStep,EulerStep

from pysph.sph.equation import Group
from pysph.sph.basic_equations import XSPHCorrection, ContinuityEquation
from pysph.sph.wc.basic import TaitEOS, MomentumEquation

In [6]:
from pysph.sph.integrator_step import IntegratorStep
class WCSPHStep2(IntegratorStep):
    def initialize(self, d_idx, d_x0, d_y0, d_z0, d_x, d_y, d_z,
                   d_u0, d_v0, d_w0, d_u, d_v, d_w, d_rho0, d_rho):
        d_x0[d_idx] = d_x[d_idx]
        d_y0[d_idx] = d_y[d_idx]
        d_z0[d_idx] = d_z[d_idx]

        d_u0[d_idx] = d_u[d_idx]
        d_v0[d_idx] = d_v[d_idx]
        d_w0[d_idx] = d_w[d_idx]

        d_rho0[d_idx] = d_rho[d_idx]

    def stage1(self, d_idx, d_x0, d_y0, d_z0, d_x, d_y, d_z,
                   d_u0, d_v0, d_w0, d_u, d_v, d_w, d_rho0, d_rho, d_au, d_av,
                   d_aw, d_ax, d_ay, d_az, d_arho, dt):
        dtb2 = 0.5*dt
        d_u[d_idx] = d_u0[d_idx] + dtb2*d_au[d_idx]
        d_v[d_idx] = d_v0[d_idx] + dtb2*d_av[d_idx]
        d_w[d_idx] = d_w0[d_idx] + dtb2*d_aw[d_idx]

        d_x[d_idx] = d_x0[d_idx] + dtb2 * d_ax[d_idx]
        d_y[d_idx] = d_y0[d_idx] + dtb2 * d_ay[d_idx]
        d_z[d_idx] = d_z0[d_idx] + dtb2 * d_az[d_idx]

        # Update densities and smoothing lengths from the accelerations
        d_rho[d_idx] = d_rho0[d_idx] + dtb2 * d_arho[d_idx]


    def stage2(self, d_idx, d_x0, d_y0, d_z0, d_x, d_y, d_z,
                   d_u0, d_v0, d_w0, d_u, d_v, d_w, d_rho0, d_rho, d_au, d_av,
                   d_aw, d_ax, d_ay, d_az, d_arho, dt):

        d_u[d_idx] = d_u0[d_idx] + dt*d_au[d_idx]
        d_v[d_idx] = d_v0[d_idx] + dt*d_av[d_idx]
        d_w[d_idx] = d_w0[d_idx] + dt*d_aw[d_idx]

        d_x[d_idx] = d_x0[d_idx] + dt * d_ax[d_idx]
        d_y[d_idx] = d_y0[d_idx] + dt * d_ay[d_idx]
        d_z[d_idx] = d_z0[d_idx] + dt * d_az[d_idx]

        # Update densities and smoothing lengths from the accelerations
        d_rho[d_idx] = d_rho0[d_idx] + dt * d_arho[d_idx]
        

In [8]:
class Test(Application):
    def initialize(self):
        import json
        with open(param_path, mode='r') as f:
            par = json.load(f)        
#         self.co = 1400.0
#         self.ro = 1.0
#         self.hdx = 1.3
#         self.dx = 0.025
#         self.alpha = 0.1
#         self.dt = 5e-6
#         self.tf = 0.0076
        self.co = par['co']
        self.ro = par['ro']
        self.hdx = par['hdx']
        self.dx = par['dx']
        self.alpha = par['alpha']
        self.dt = par['dt']
        self.tf = par['tf']
        
    def create_solver(self):
        kernel = CubicSpline(dim=2)

        integrator = EPECIntegrator(fluid=WCSPHStep())

        solver = Solver(kernel=kernel, dim=2, integrator=integrator,
                        dt=self.dt, tf=self.tf,pfreq=10)

        return solver

    def create_equations(self):
        equations = [
            Group(
                equations=[
                    TaitEOS(
                        dest='fluid', sources=None, 
                        rho0=self.ro,c0=self.co, gamma=7.0
                    ),
                ],
                real=False
            ),
            Group(equations=[
                ContinuityEquation(dest='fluid',  sources=['fluid']),
                MomentumEquation(
                    dest='fluid', sources=['fluid'],
                    alpha=self.alpha, beta=0.0, c0=self.co
                ),
                XSPHCorrection(
                    dest='fluid', sources=['fluid'], eps=0.0)
            ]),
        ]
        return equations    

    def create_particles(self):
        """Create the circular patch of fluid."""
        name = 'fluid'
        dx=self.dx
        hdx=self.hdx
        ro=self.ro

        x, y = mgrid[-1.05:1.05+1e-4:dx, -1.05:1.05+1e-4:dx]
        x = x.ravel()
        y = y.ravel()

        m = ones_like(x)*dx*dx*ro
        h = ones_like(x)*hdx*dx
        rho = ones_like(x) *ro
        u = -100*x
        v = 100*y

        # remove particles outside the circle
        indices = []
        for i in range(len(x)):
            if sqrt(x[i]*x[i] + y[i]*y[i]) - 1 > 1e-10:
                indices.append(i)

        pa = get_particle_array_wcsph(x=x, y=y, m=m, rho=rho, h=h, u=u, v=v,
                                name=name)
        pa.remove_particles(indices)

        print("Elliptical drop :: %d particles"
              % (pa.get_number_of_particles()))

        pa.set_output_arrays(['x','y','u', 'v', 'rho', 'h', 'p', 'pid', 'tag', 'gid'])
        return [pa]

if __name__ == '__main__':
    app = Test(fname=model_path)
    app.run()

Elliptical drop :: 5025 particles
Generating output in /home/ec2-user/SageMaker/PySPH/model/_output
Precompiled code from: /home/ec2-user/.pysph/source/py3.6-linux-x86_64/m_3cb03a7e4beb4aa5a27ae0b5b97cbf8a.pyx


AttributeError: type object 'm_3cb03a7e4beb4aa5a27ae0b5b97cbf8a.CubicSpline' has no attribute '__reduce_cython__'

In [1]:
!conda install -c conda-forge ipympl -y
!conda install -c conda-forge widgetsnbextension -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.8

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python2

  added / updated specs: 
    - ipympl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipympl-0.2.1               |        py27_1001         1.1 MB  conda-forge
    certifi-2019.3.9           |           py27_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.2 MB

The following NEW packages will be INSTALLED:

    ipympl:          0.2.1-py27_1001   conda-forge

The following packages will be UPDATED:

    ca-certificates: 2019.1.23-0                   --> 2019.3.9-hecc5488_0 conda-forge
    certi